In [1]:
import pandas as pd
import csv
import collections
import re
import pickle
import numpy as np

In [2]:
fileNeop = "../data/RTRT_NEOPLASI_corr.csv"
fileIsto = "../data/ISTOLOGIE_corr.csv" 
fileIcdo = "icdo-3-dict-en.p"

fileSaveCorpus = "corpusTemporalV2b/corpusTemporal.p"
fileSaveValues = "corpusTemporalV2b/valuesTemporalSede1.p"
fileSaveValuesM = "corpusTemporalV2b/valuesTemporalMorfo1.p"
fileSaveTrainWordVectors = "corpusTemporalV2b/trainWordVectors.txt"

In [3]:
encoding = 'iso-8859-1'

In [4]:
icdo = pickle.load(open(fileIcdo, 'rb'))

# Load neop

In [5]:
dfNeop = pd.read_csv(fileNeop, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False,
                     usecols = ['id_neopl', 'sede_icdo3',
                                'morfologia_icdo3', 'inserimento_data_neoplasi'],
                     dtype={'id_neopl':pd.np.float64, 'sede_icdo3':pd.np.str, 
                            'morfologia_icdo3':pd.np.str, 'inserimento_data_neoplasi':pd.np.str},
                    #parse_dates=['inserimento_data_neoplasi'], 
                    #date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
                    )

In [6]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [7]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [8]:
dfNeop = dfNeop.dropna('index')

In [9]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [10]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [11]:
dfNeop['inserimento_data_neoplasi'] =  pd.to_datetime(dfNeop['inserimento_data_neoplasi'], format='%Y-%m-%d %H:%M:%S')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [13]:
dfNeop.dtypes

id_neopl                            float64
inserimento_data_neoplasi    datetime64[ns]
sede_icdo3                           object
morfologia_icdo3                     object
dtype: object

In [14]:
dfNeop = dfNeop.sort_values(by=['inserimento_data_neoplasi'])

In [15]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
107276,108110.0,1990-04-20 00:00:00,C443,80003
78576,79172.0,1990-04-20 00:00:00,C341,80003
78595,79191.0,1990-04-24 00:00:00,C329,80703
78602,79198.0,1990-04-24 00:00:00,C509,85203
78601,79197.0,1990-04-24 00:00:00,C710,94003
...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001
295873,346866.0,2014-03-10 12:26:26,C169,81403


remove invalid sites

In [16]:
dfNeop['correctSede'] = dfNeop.sede_icdo3.map(lambda s: len(s)==4 and s[0] == "C" and int(s[1:3]) in icdo.keys())

In [17]:
dfNeop = dfNeop[dfNeop.correctSede]

In [18]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede
107276,108110.0,1990-04-20 00:00:00,C443,80003,True
78576,79172.0,1990-04-20 00:00:00,C341,80003,True
78595,79191.0,1990-04-24 00:00:00,C329,80703,True
78602,79198.0,1990-04-24 00:00:00,C509,85203,True
78601,79197.0,1990-04-24 00:00:00,C710,94003,True
...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True
309849,346856.0,2014-02-14 11:53:40,C424,97651,True
309850,346857.0,2014-02-14 12:00:33,C619,80001,True
295873,346866.0,2014-03-10 12:26:26,C169,81403,True


In [19]:
dfNeop['y'] = dfNeop.sede_icdo3.map(lambda s: int(s[1:3]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dfNeop['yM'] = dfNeop.morfologia_icdo3.map(lambda m: int(m[:4]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,yM
107276,108110.0,1990-04-20 00:00:00,C443,80003,True,44,8000
78576,79172.0,1990-04-20 00:00:00,C341,80003,True,34,8000
78595,79191.0,1990-04-24 00:00:00,C329,80703,True,32,8070
78602,79198.0,1990-04-24 00:00:00,C509,85203,True,50,8520
78601,79197.0,1990-04-24 00:00:00,C710,94003,True,71,9400
...,...,...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True,50,8500
309849,346856.0,2014-02-14 11:53:40,C424,97651,True,42,9765
309850,346857.0,2014-02-14 12:00:33,C619,80001,True,61,8000
295873,346866.0,2014-03-10 12:26:26,C169,81403,True,16,8140


# Load isto

In [22]:
dfIsto = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [23]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [24]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfIsto['notizie'] = dfIsto.notizie.map(nullToEmpty)
dfIsto['diagnosi'] = dfIsto.diagnosi.map(nullToEmpty)
dfIsto['macroscopia'] = dfIsto.macroscopia.map(nullToEmpty)

In [25]:
dfIsto = dfIsto.dropna('index')

In [26]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril..."
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.
...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...


In [27]:
dfIsto['text'] = dfIsto[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
dfIsto['text'] = dfIsto['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
dfIsto['text'] = dfIsto['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...
...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...


In [31]:
dfIsto['correctText'] = dfIsto.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
dfIsto = dfIsto[dfIsto.correctText]

In [33]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [34]:
dfIsto = dfIsto.drop_duplicates(['text'])

In [35]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [36]:
dfIsto = dfIsto.drop_duplicates(['id_neopl'], keep=False)

In [37]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
684,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


# Merge

In [38]:
dfMerge = pd.merge(dfIsto, dfNeop, on='id_neopl', validate="one_to_one")

In [39]:
dfMerge

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,yM
0,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True,2008-03-06 14:14:39,C449,80903,True,44,8090
1,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True,2012-11-14 10:01:56,C449,80903,True,44,8090
2,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True,2007-02-09 10:07:13,C779,96513,True,77,9651
3,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True,2012-10-10 10:19:28,C209,81401,True,20,8140
4,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True,2012-10-10 14:21:28,C449,80903,True,44,8090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85170,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True,2013-10-18 16:23:09,C443,80903,True,44,8090
85171,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True,2012-09-10 16:00:08,C186,80003,True,18,8000
85172,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True,2012-10-15 11:56:30,C329,80703,True,32,8070
85173,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True,2012-11-12 10:35:31,C447,87432,True,44,8743


# Unlabelled

In [40]:
dfUn = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [41]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [42]:
dfUn = dfUn[~dfUn.id_fonte.isin(dfMerge.id_fonte)]

In [43]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [44]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfUn['notizie'] = dfUn.notizie.map(nullToEmpty)
dfUn['diagnosi'] = dfUn.diagnosi.map(nullToEmpty)
dfUn['macroscopia'] = dfUn.macroscopia.map(nullToEmpty)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [45]:
dfUn['text'] = dfUn[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
dfUn['text'] = dfUn['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
dfUn['text'] = dfUn['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
dfUn['correctText'] = dfUn.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
dfUn = dfUn[dfUn.correctText]

In [50]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...,True
2,3.0,NaN,,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM.",ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...,True
3,4.0,NaN,,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...,True
4,5.0,NaN,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
5,6.0,NaN,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO (vedi es...,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...,True
...,...,...,...,...,...,...,...
1592380,1792447.0,NaN,,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...,"RESIDUE , MINIME ESPRESSIONI DI MALIGNITA ' EP...",True
1592381,1792448.0,NaN,,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,True
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...,True
1592383,1792450.0,NaN,,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...,True


In [51]:
dfUn = dfUn[~dfUn.text.isin(dfMerge.text)]

In [52]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...,True
2,3.0,NaN,,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM.",ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...,True
3,4.0,NaN,,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...,True
4,5.0,NaN,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
5,6.0,NaN,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO (vedi es...,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...,True
...,...,...,...,...,...,...,...
1592379,1792446.0,NaN,,ADENOMA TUBULARE INTESTINALE DI ALTO GRADO; BA...,"Polipo del colon ascendente (cm 0,7). (eseguit...",ADENOMA TUBULARE INTESTINALE DI ALTO GRADO ; B...,True
1592381,1792448.0,NaN,,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,True
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...,True
1592383,1792450.0,NaN,,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...,True


# Final

In [53]:
dfFinal = dfMerge[['inserimento_data_neoplasi','text', 'y', 'yM']]

In [54]:
dfFinal = dfFinal.rename(columns={"inserimento_data_neoplasi": "data"})

In [55]:
dfFinal = dfFinal.sort_values(by=['data'])

In [56]:
dfFinal

,data,text,y,yM
61615,1992-04-13 00:00:00,METASTASI EPATICHE DA SOSPETTA NEOPLASIA COLON...,16,8140
70999,1992-04-15 00:00:00,PSA : 1160 ; AREA IPOECOGENA LOBO SINISTRO ; I...,75,8140
46748,1996-03-13 00:00:00,CARCINOMA DUTTALE INFILTRANTE ( NOS ) ( G2 ) D...,50,8501
17030,1997-03-04 00:00:00,LESIONE PIGMENTATA DISCROMICA E PALPABILE OD :...,44,8743
67580,1997-09-15 00:00:00,AREE DI ADENOCARCINOMA PROSTATICO A MEDIO GRAD...,23,8560
...,...,...,...,...
84732,2014-02-13 08:51:32,CARCINOMA DUTTALE INFILTRANTE ( G2 ) DELLA MAM...,50,8500
47068,2014-02-13 11:20:50,"CARCINOMA DUTTALE IN SITU NAS , PRESENZA DI CA...",50,8500
78074,2014-02-14 12:00:33,1 - AGOBIOPSIA PROSTATICA DELLA LUNGHEZZA COMP...,61,8000
1000,2014-03-10 12:26:26,CA . GASTRICO . ADENOCARCINOMA GASTRICO MODERA...,16,8140


In [57]:
dfUnlabelled = dfUn[['text']]

In [58]:
dfUnlabelled

,text
1,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...
2,ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...
3,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...
4,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
5,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...
...,...
1592379,ADENOMA TUBULARE INTESTINALE DI ALTO GRADO ; B...
1592381,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...
1592382,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...
1592383,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...


In [59]:
c = collections.Counter(dfFinal.y)
c.most_common()

[(44, 20994),
 (50, 9510),
 (18, 9247),
 (61, 6738),
 (67, 5514),
 (34, 4511),
 (16, 3788),
 (42, 3107),
 (20, 2838),
 (64, 1953),
 (54, 1895),
 (53, 1729),
 (56, 1187),
 (32, 935),
 (73, 928),
 (25, 846),
 (19, 782),
 (77, 703),
 (71, 676),
 (22, 624),
 (49, 554),
 (80, 482),
 (15, 323),
 (24, 317),
 (26, 304),
 (62, 297),
 (70, 296),
 (17, 246),
 (2, 241),
 (51, 238),
 (23, 228),
 (21, 224),
 (6, 217),
 (38, 200),
 (75, 171),
 (66, 157),
 (48, 142),
 (65, 138),
 (7, 134),
 (9, 132),
 (0, 130),
 (30, 104),
 (72, 104),
 (68, 104),
 (11, 98),
 (60, 84),
 (40, 76),
 (41, 76),
 (69, 74),
 (57, 66),
 (5, 61),
 (10, 59),
 (8, 59),
 (31, 57),
 (55, 53),
 (74, 47),
 (1, 47),
 (52, 44),
 (12, 43),
 (4, 43),
 (13, 36),
 (3, 34),
 (37, 30),
 (14, 24),
 (76, 24),
 (47, 22),
 (63, 19),
 (33, 6),
 (58, 4),
 (39, 1)]

In [60]:
c = collections.Counter(dfFinal.yM)
c.most_common()

[(8140, 20757),
 (8090, 14105),
 (8070, 7737),
 (8500, 5829),
 (8130, 3979),
 (8010, 2951),
 (8120, 1642),
 (8312, 1410),
 (8000, 1292),
 (8480, 1259),
 (8261, 1259),
 (8380, 1169),
 (8743, 1128),
 (8520, 929),
 (8144, 791),
 (9732, 758),
 (9680, 707),
 (8720, 636),
 (8260, 554),
 (8041, 528),
 (8522, 470),
 (8081, 433),
 (8550, 408),
 (8170, 384),
 (8211, 382),
 (8890, 378),
 (9440, 359),
 (8441, 331),
 (8490, 323),
 (9591, 299),
 (8145, 292),
 (9690, 288),
 (8201, 286),
 (8077, 275),
 (8931, 235),
 (8742, 232),
 (8046, 230),
 (8340, 226),
 (8001, 224),
 (8310, 213),
 (9670, 210),
 (9590, 209),
 (8240, 194),
 (8503, 190),
 (8076, 188),
 (8560, 171),
 (8050, 168),
 (9823, 164),
 (8323, 163),
 (8020, 154),
 (9061, 154),
 (8721, 153),
 (8094, 147),
 (8071, 144),
 (8936, 144),
 (8072, 143),
 (8501, 137),
 (9960, 135),
 (9989, 135),
 (8460, 129),
 (9699, 123),
 (8246, 121),
 (8210, 121),
 (9140, 109),
 (9532, 102),
 (9530, 97),
 (8263, 94),
 (8771, 93),
 (9667, 92),
 (9663, 89),
 (8832, 89

In [61]:
testLen = int(len(dfFinal)/100*20)

In [62]:
dfTest = dfFinal.tail(testLen)

In [63]:
dfTrain = dfFinal.head(len(dfFinal) - testLen)

In [64]:
dfValid = dfTrain.tail(testLen)

In [65]:
dfTrain = dfTrain.head(len(dfTrain) - testLen)

In [66]:
dfTrain

,data,text,y,yM
61615,1992-04-13 00:00:00,METASTASI EPATICHE DA SOSPETTA NEOPLASIA COLON...,16,8140
70999,1992-04-15 00:00:00,PSA : 1160 ; AREA IPOECOGENA LOBO SINISTRO ; I...,75,8140
46748,1996-03-13 00:00:00,CARCINOMA DUTTALE INFILTRANTE ( NOS ) ( G2 ) D...,50,8501
17030,1997-03-04 00:00:00,LESIONE PIGMENTATA DISCROMICA E PALPABILE OD :...,44,8743
67580,1997-09-15 00:00:00,AREE DI ADENOCARCINOMA PROSTATICO A MEDIO GRAD...,23,8560
...,...,...,...,...
51629,2010-12-30 11:37:10,COMPONENTE MONOCLONALE IGA . OD : MIELOMA . MI...,42,9732
49858,2010-12-30 11:37:10,CARCINOMA BASOCELLULARE INFILTRANTE IL DERMA R...,44,8090
49859,2010-12-30 11:37:10,1 - ADENOMA TUBULO - VILLOSO INTESTINALE CON D...,18,8263
49860,2010-12-30 11:37:10,CARCINOMA SQUAMOCELLULARE BEN DIFFERENZIATO IN...,44,8070


In [67]:
dfValid

,data,text,y,yM
49874,2010-12-30 11:37:10,CARCINOMA BASOSQUAMOCELLULARE ULCERATO INFILTR...,44,8070
48566,2010-12-30 11:37:10,CARCINOMA A CELLULE BASALI . LESIONE NON COMPR...,44,8090
45980,2010-12-30 11:37:10,ISOLATI FRAMMENTI RIFERIBILI AD ADENOMA TUBULA...,20,8211
47380,2010-12-30 11:37:10,CARCINOMA BASOCELLULARE DI TIPO SUPERFICIALE ....,44,8090
48379,2010-12-30 11:37:10,RETTORRAGIA . ADENOMA TUBULO - VILLOSO INTESTI...,20,8140
...,...,...,...,...
75683,2012-03-06 13:09:47,RESIDUE ESPRESSIONI DI CARCINOMA BASOCELLULARE...,44,8090
79907,2012-03-06 13:09:59,CARCINOMA A CELLULE CHIARE RENALE ( FUHRMAN : ...,64,8312
74879,2012-03-06 13:10:20,NEVO MELANOCITICO COMPOSTO DI TIPO CONGENITO A...,44,8093
74380,2012-03-06 13:10:48,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,18,8140


In [68]:
dfTest

,data,text,y,yM
78863,2012-03-06 13:11:25,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI ( A...,44,8090
75006,2012-03-06 13:11:45,FRAMMENTI ISTOLOGICI RIFERIBILI AD ADENOCARCIN...,18,8140
75935,2012-03-06 13:11:55,CARCINOMA INFILTRANTE DI TIPO COLLOIDE EXTRACE...,50,8480
79952,2012-03-06 13:12:18,PRESENZA DI UN SINGOLO FOCOLAIO DI ADENOCARCIN...,61,8140
79484,2012-03-06 13:12:31,ADENOCARCINOMA ENDOMETRIALE DI BASSO GRADO ( G...,54,8380
...,...,...,...,...
84732,2014-02-13 08:51:32,CARCINOMA DUTTALE INFILTRANTE ( G2 ) DELLA MAM...,50,8500
47068,2014-02-13 11:20:50,"CARCINOMA DUTTALE IN SITU NAS , PRESENZA DI CA...",50,8500
78074,2014-02-14 12:00:33,1 - AGOBIOPSIA PROSTATICA DELLA LUNGHEZZA COMP...,61,8000
1000,2014-03-10 12:26:26,CA . GASTRICO . ADENOCARCINOMA GASTRICO MODERA...,16,8140


### remove y not in train valid test

In [110]:
cTrain = collections.Counter(dfTrain.y)
cTrain.most_common()

[(44, 11409),
 (50, 5953),
 (18, 5187),
 (61, 3968),
 (67, 3338),
 (34, 3151),
 (16, 2397),
 (42, 1978),
 (20, 1699),
 (64, 1247),
 (54, 1066),
 (53, 880),
 (56, 687),
 (32, 622),
 (73, 612),
 (25, 567),
 (19, 507),
 (71, 443),
 (22, 383),
 (49, 285),
 (80, 279),
 (70, 222),
 (24, 212),
 (15, 212),
 (62, 183),
 (77, 170),
 (6, 151),
 (51, 150),
 (23, 146),
 (2, 146),
 (17, 139),
 (75, 130),
 (38, 113),
 (21, 113),
 (66, 106),
 (68, 95),
 (0, 88),
 (72, 88),
 (48, 80),
 (26, 79),
 (65, 78),
 (9, 77),
 (7, 76),
 (11, 58),
 (30, 55),
 (60, 51),
 (10, 48),
 (57, 47),
 (40, 45),
 (69, 41),
 (31, 40),
 (8, 39),
 (55, 38),
 (5, 34),
 (1, 34),
 (41, 32),
 (12, 32),
 (4, 27),
 (52, 23),
 (3, 22),
 (13, 22),
 (76, 17),
 (74, 16),
 (14, 15),
 (63, 14),
 (47, 11),
 (37, 8),
 (33, 3)]

In [111]:
cValid = collections.Counter(dfValid.y)
cValid.most_common()

[(44, 5231),
 (50, 1900),
 (18, 1889),
 (61, 1866),
 (34, 827),
 (16, 717),
 (20, 553),
 (54, 456),
 (42, 394),
 (67, 371),
 (64, 361),
 (53, 201),
 (56, 184),
 (77, 182),
 (19, 159),
 (73, 148),
 (32, 146),
 (26, 115),
 (25, 104),
 (71, 103),
 (49, 100),
 (22, 97),
 (15, 65),
 (2, 61),
 (80, 54),
 (21, 52),
 (17, 49),
 (24, 44),
 (51, 42),
 (62, 40),
 (6, 37),
 (38, 33),
 (23, 33),
 (9, 26),
 (7, 25),
 (0, 24),
 (41, 22),
 (30, 20),
 (66, 19),
 (48, 17),
 (60, 17),
 (65, 16),
 (11, 15),
 (40, 12),
 (57, 12),
 (52, 11),
 (75, 11),
 (8, 11),
 (5, 10),
 (70, 9),
 (4, 9),
 (69, 8),
 (13, 8),
 (12, 7),
 (74, 7),
 (55, 6),
 (3, 6),
 (10, 5),
 (1, 5),
 (76, 4),
 (72, 3),
 (63, 3),
 (31, 3),
 (47, 3),
 (14, 2),
 (37, 1),
 (33, 1),
 (68, 1)]

In [112]:
cTest = collections.Counter(dfTest.y)
cTest.most_common()

[(44, 4205),
 (18, 2151),
 (67, 1796),
 (50, 1592),
 (61, 903),
 (16, 668),
 (53, 642),
 (42, 586),
 (20, 579),
 (34, 495),
 (54, 357),
 (64, 321),
 (56, 297),
 (77, 258),
 (25, 166),
 (32, 163),
 (80, 144),
 (73, 143),
 (49, 117),
 (22, 116),
 (19, 115),
 (26, 108),
 (24, 59),
 (71, 55),
 (21, 53),
 (17, 50),
 (23, 46),
 (15, 46),
 (51, 45),
 (65, 44),
 (48, 42),
 (62, 41),
 (2, 34),
 (38, 34),
 (66, 32),
 (70, 28),
 (9, 27),
 (30, 26),
 (6, 25),
 (7, 25),
 (11, 24),
 (75, 19),
 (0, 18),
 (5, 16),
 (41, 16),
 (31, 14),
 (60, 13),
 (40, 13),
 (69, 10),
 (8, 8),
 (1, 7),
 (52, 7),
 (55, 7),
 (4, 7),
 (14, 7),
 (57, 6),
 (10, 6),
 (68, 6),
 (13, 6),
 (3, 5),
 (47, 5),
 (12, 4),
 (72, 3),
 (76, 3),
 (33, 2),
 (37, 2),
 (63, 1),
 (74, 1)]

In [113]:
cTest[44]

4205

In [114]:
toRemove = set()
for i in cTrain.keys():
    if not i in cValid.keys() or not i in cTest.keys():
        toRemove.add(i)
    

In [115]:
toRemove

set()

In [116]:
for i in cValid.keys():
    if not i in cTest.keys() or not i in cTrain.keys():
        toRemove.add(i)

In [117]:
toRemove

set()

In [118]:
for i in cTest.keys():
    if not i in cTrain.keys() or not i in cValid.keys():
        toRemove.add(i)

In [119]:
toRemove

set()

In [79]:
dfTrain = dfTrain[~dfTrain.y.isin(toRemove)]

In [80]:
dfValid = dfValid[~dfValid.y.isin(toRemove)]

In [81]:
dfTest = dfTest[~dfTest.y.isin(toRemove)]

### remove yM not in train valid test

In [82]:
cTrain = collections.Counter(dfTrain.yM)
cTrain.most_common()

[(8140, 12181),
 (8090, 7443),
 (8070, 4764),
 (8500, 3745),
 (8130, 2468),
 (8010, 1846),
 (8120, 1014),
 (8312, 958),
 (8261, 914),
 (8480, 876),
 (8743, 801),
 (8380, 695),
 (8520, 609),
 (8144, 574),
 (8000, 566),
 (9732, 464),
 (9680, 448),
 (8720, 402),
 (8041, 376),
 (8260, 343),
 (8522, 306),
 (8211, 278),
 (9440, 278),
 (8170, 251),
 (9591, 234),
 (8201, 222),
 (8441, 213),
 (8890, 201),
 (8490, 187),
 (8550, 178),
 (9690, 175),
 (8145, 173),
 (8340, 157),
 (9590, 146),
 (8081, 142),
 (9670, 141),
 (8050, 138),
 (8560, 131),
 (8240, 126),
 (8721, 121),
 (8503, 114),
 (8310, 114),
 (8210, 114),
 (8501, 113),
 (9823, 113),
 (8020, 113),
 (8742, 111),
 (8323, 107),
 (9061, 107),
 (8076, 102),
 (8071, 94),
 (8072, 91),
 (9532, 87),
 (8460, 86),
 (9663, 85),
 (8094, 85),
 (8771, 84),
 (8046, 84),
 (8936, 83),
 (9960, 78),
 (9530, 78),
 (9140, 66),
 (8200, 66),
 (9531, 63),
 (8931, 61),
 (8263, 60),
 (9863, 56),
 (9765, 56),
 (8832, 55),
 (8850, 55),
 (9380, 51),
 (9650, 50),
 (9050

In [83]:
cValid = collections.Counter(dfValid.yM)
cValid.most_common()

[(8140, 4522),
 (8090, 4072),
 (8070, 1467),
 (8500, 1061),
 (8010, 544),
 (8000, 381),
 (8380, 316),
 (8312, 264),
 (8480, 228),
 (8130, 220),
 (8520, 194),
 (8550, 177),
 (9732, 174),
 (8261, 169),
 (8001, 126),
 (8522, 113),
 (8046, 108),
 (8120, 108),
 (8144, 108),
 (9680, 106),
 (8081, 104),
 (8041, 94),
 (8260, 90),
 (8145, 80),
 (8211, 78),
 (8490, 65),
 (8076, 63),
 (8170, 63),
 (8931, 62),
 (9440, 60),
 (8441, 55),
 (9690, 49),
 (8890, 47),
 (8310, 47),
 (8720, 46),
 (8340, 46),
 (8240, 44),
 (8323, 42),
 (8077, 41),
 (8743, 40),
 (8503, 38),
 (8742, 36),
 (8071, 35),
 (8092, 32),
 (8460, 32),
 (8072, 31),
 (9667, 31),
 (8246, 29),
 (8201, 29),
 (9140, 28),
 (9061, 25),
 (8094, 24),
 (9734, 24),
 (9591, 23),
 (9699, 23),
 (8020, 23),
 (8050, 22),
 (8042, 22),
 (9863, 21),
 (9380, 21),
 (9823, 20),
 (9989, 19),
 (8560, 19),
 (8936, 19),
 (9220, 18),
 (8832, 17),
 (8230, 17),
 (8095, 16),
 (8263, 16),
 (8093, 16),
 (9960, 16),
 (9050, 15),
 (9962, 14),
 (8830, 14),
 (8200, 14),


In [84]:
cTest = collections.Counter(dfTest.yM)
cTest.most_common()

[(8140, 4054),
 (8090, 2590),
 (8070, 1505),
 (8130, 1291),
 (8500, 1023),
 (8010, 561),
 (8120, 520),
 (8000, 345),
 (8743, 287),
 (8077, 195),
 (8312, 188),
 (8720, 188),
 (8081, 187),
 (8261, 176),
 (8380, 158),
 (8480, 155),
 (9680, 153),
 (8890, 130),
 (8520, 126),
 (8260, 121),
 (9732, 120),
 (8931, 112),
 (8144, 109),
 (8742, 85),
 (9989, 78),
 (8001, 75),
 (8490, 71),
 (8170, 70),
 (9690, 64),
 (8441, 63),
 (8041, 58),
 (9670, 56),
 (9699, 55),
 (8550, 53),
 (8310, 52),
 (8522, 51),
 (9590, 50),
 (8246, 46),
 (8936, 42),
 (9591, 42),
 (9960, 41),
 (8440, 41),
 (8145, 39),
 (8503, 38),
 (8046, 38),
 (8094, 38),
 (8201, 35),
 (9823, 31),
 (8721, 30),
 (9667, 30),
 (8400, 29),
 (9673, 28),
 (8211, 26),
 (8744, 24),
 (8240, 24),
 (8076, 23),
 (8830, 23),
 (8340, 23),
 (9061, 22),
 (9980, 22),
 (8560, 21),
 (8072, 21),
 (9861, 21),
 (9961, 21),
 (9440, 21),
 (9962, 21),
 (8160, 20),
 (8800, 19),
 (8390, 19),
 (8850, 19),
 (8020, 18),
 (9765, 18),
 (8263, 18),
 (8832, 17),
 (9140, 15

In [85]:
toRemove = set()
for i in cTrain.keys():
    if not i in cValid.keys() or not i in cTest.keys():
        toRemove.add(i)
    

In [86]:
toRemove

{8003,
 8013,
 8030,
 8031,
 8033,
 8040,
 8043,
 8045,
 8052,
 8053,
 8073,
 8074,
 8075,
 8080,
 8091,
 8100,
 8110,
 8122,
 8124,
 8141,
 8150,
 8154,
 8156,
 8190,
 8220,
 8221,
 8243,
 8244,
 8245,
 8251,
 8262,
 8270,
 8272,
 8331,
 8332,
 8337,
 8341,
 8350,
 8370,
 8400,
 8402,
 8404,
 8450,
 8471,
 8481,
 8502,
 8512,
 8513,
 8521,
 8523,
 8524,
 8530,
 8540,
 8561,
 8570,
 8573,
 8580,
 8582,
 8583,
 8584,
 8585,
 8650,
 8690,
 8691,
 8692,
 8693,
 8730,
 8740,
 8741,
 8745,
 8750,
 8770,
 8780,
 8790,
 8803,
 8804,
 8805,
 8814,
 8821,
 8825,
 8840,
 8853,
 8855,
 8858,
 8891,
 8892,
 8894,
 8895,
 8896,
 8900,
 8901,
 8910,
 8920,
 8930,
 8932,
 8940,
 8941,
 8950,
 8960,
 8963,
 8982,
 8990,
 9011,
 9013,
 9041,
 9043,
 9051,
 9053,
 9060,
 9062,
 9063,
 9071,
 9083,
 9101,
 9110,
 9121,
 9133,
 9161,
 9181,
 9200,
 9381,
 9382,
 9383,
 9384,
 9390,
 9392,
 9394,
 9411,
 9413,
 9420,
 9421,
 9441,
 9442,
 9471,
 9490,
 9500,
 9505,
 9506,
 9510,
 9522,
 9533,
 9534,
 9537,

In [87]:
len(toRemove)

183

In [88]:
for i in cValid.keys():
    if not i in cTest.keys() or not i in cTrain.keys():
        toRemove.add(i)

In [89]:
toRemove

{8003,
 8013,
 8030,
 8031,
 8033,
 8040,
 8043,
 8045,
 8052,
 8053,
 8073,
 8074,
 8075,
 8080,
 8083,
 8091,
 8097,
 8100,
 8110,
 8122,
 8124,
 8141,
 8150,
 8154,
 8156,
 8190,
 8220,
 8221,
 8243,
 8244,
 8245,
 8251,
 8262,
 8270,
 8272,
 8314,
 8315,
 8331,
 8332,
 8337,
 8341,
 8350,
 8370,
 8400,
 8402,
 8404,
 8407,
 8450,
 8453,
 8471,
 8481,
 8482,
 8502,
 8512,
 8513,
 8521,
 8523,
 8524,
 8525,
 8530,
 8540,
 8561,
 8570,
 8573,
 8580,
 8582,
 8583,
 8584,
 8585,
 8650,
 8690,
 8691,
 8692,
 8693,
 8727,
 8730,
 8740,
 8741,
 8745,
 8750,
 8770,
 8780,
 8790,
 8803,
 8804,
 8805,
 8814,
 8821,
 8825,
 8831,
 8840,
 8853,
 8855,
 8858,
 8891,
 8892,
 8894,
 8895,
 8896,
 8897,
 8900,
 8901,
 8910,
 8920,
 8930,
 8932,
 8940,
 8941,
 8950,
 8960,
 8963,
 8973,
 8982,
 8990,
 9011,
 9013,
 9041,
 9043,
 9051,
 9053,
 9060,
 9062,
 9063,
 9065,
 9071,
 9083,
 9101,
 9110,
 9121,
 9133,
 9161,
 9181,
 9183,
 9200,
 9210,
 9231,
 9381,
 9382,
 9383,
 9384,
 9390,
 9392,
 9394,

In [90]:
len(toRemove)

203

In [91]:
for i in cTest.keys():
    if not i in cTrain.keys() or not i in cValid.keys():
        toRemove.add(i)

In [92]:
toRemove

{8002,
 8003,
 8013,
 8030,
 8031,
 8033,
 8040,
 8043,
 8045,
 8052,
 8053,
 8073,
 8074,
 8075,
 8080,
 8083,
 8091,
 8097,
 8100,
 8110,
 8122,
 8124,
 8141,
 8150,
 8154,
 8156,
 8162,
 8190,
 8220,
 8221,
 8243,
 8244,
 8245,
 8251,
 8262,
 8270,
 8272,
 8314,
 8315,
 8319,
 8322,
 8331,
 8332,
 8337,
 8341,
 8342,
 8350,
 8370,
 8400,
 8402,
 8403,
 8404,
 8407,
 8413,
 8450,
 8453,
 8471,
 8481,
 8482,
 8502,
 8512,
 8513,
 8514,
 8521,
 8523,
 8524,
 8525,
 8530,
 8540,
 8561,
 8570,
 8573,
 8580,
 8582,
 8583,
 8584,
 8585,
 8621,
 8640,
 8650,
 8690,
 8691,
 8692,
 8693,
 8727,
 8730,
 8740,
 8741,
 8745,
 8750,
 8770,
 8780,
 8790,
 8803,
 8804,
 8805,
 8814,
 8821,
 8823,
 8825,
 8831,
 8834,
 8840,
 8853,
 8855,
 8858,
 8891,
 8892,
 8894,
 8895,
 8896,
 8897,
 8900,
 8901,
 8910,
 8920,
 8930,
 8932,
 8940,
 8941,
 8950,
 8960,
 8963,
 8973,
 8982,
 8990,
 9011,
 9013,
 9041,
 9043,
 9051,
 9053,
 9060,
 9062,
 9063,
 9065,
 9071,
 9083,
 9101,
 9110,
 9121,
 9133,
 9161,

In [93]:
len(toRemove)

224

In [94]:
dfTrain = dfTrain[~dfTrain.yM.isin(toRemove)]

In [95]:
dfValid = dfValid[~dfValid.yM.isin(toRemove)]

In [96]:
dfTest = dfTest[~dfTest.yM.isin(toRemove)]

Manually checked that, after that, is still valid for sede1

In [97]:
len(dfTrain)

50284

In [98]:
len(dfValid)

16973

In [99]:
len(dfTest)

16870

In [100]:
corpus = {
    'train':{
        'text':dfTrain['text'].values,
        'sede1':dfTrain['y'].values,
        'morfo1':dfTrain['yM'].values,
    },
    'valid':{
        'text':dfValid['text'].values,
        'sede1':dfValid['y'].values,
        'morfo1':dfValid['yM'].values,
    },
    'test':{
        'text':dfTest['text'].values,
        'sede1':dfTest['y'].values,
        'morfo1':dfTest['yM'].values,
    },
}

In [101]:
count = collections.Counter(np.concatenate((corpus['train']['sede1'], corpus['valid']['sede1'], corpus['test']['sede1'])))

In [102]:
values = sorted(count.keys())

In [103]:
countM = collections.Counter(np.concatenate((corpus['train']['morfo1'], corpus['valid']['morfo1'], corpus['test']['morfo1'])))
valuesM = sorted(countM.keys())

In [104]:
len(values)

68

In [105]:
wordVectorsText = np.concatenate((dfUnlabelled.text.values, dfTrain.text.values, dfValid.text.values))

In [106]:
wordVectorsText[:10]

array(['PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE DOPO INCLUSIONE . ESAME ISTOLOGICO AL CONGELATORE .',
       'ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGMA , DIAMETRO MAGGIORE 8 MM .',
       'FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI ASPETTO LIPOMATOSO DI 3CM COMPLESSIVAMENTE .',
       'QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO COMPATIBILI CON LINFOMA GASTRICO ( VEDI ES . K 331 / 99 )',
       'QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI ES . K 209 / 99 ) COMPATIBILE CON LLC MT',
       'ESAME INTRAOPERATORIO E PER INCLUSIONE : TUMORE ADENOMATOIDE . MT - FB - FS NODO DEL DIAMETRO MASSIMO DI CM 0 , 7 .',
       "NEOPLASIA DEL POLO INFERIORE DEL RENE SINISTRO . CARCINOMA A CELLULE RENALI CHIARE . STRUTTURE DELL ' ILO INDENNI . PT2 . 2 BB RENE SINISTRO DI CM 12 * 6 * 5 CON NEOPLASIA DI CM 5 .",
       'LIPOMA . MT',
       'ELEMENTI EPITELIALI ATIPICI COMPATIBILI CON CARCINOMA . QUATTRO VETRINI .',
       'ESAME INTRAOPERATORIO E PER

In [107]:
np.random.shuffle(wordVectorsText)

In [108]:
wordVectorsText[:10]

array(["NEOFORMAZIONE A PLACCA FIANCO SX OD : EPITELIOMA BASOCELLULARE CARCINOMA A CELLULE BASALI . LA PROLIFERAZIONE NEOPLASTICA E ' CONTENUTA ENTRO I MARGINI DEL FRAMMENTO . FRAMMENTO CUTANEO DI 1 , 5 X 0 , 7 X 0 , 5 CM CON AREA DISCROMICA DI 0 , 6 X 0 , 4 CM . 1 - 3 ) SEZIONI A CROCE ( MARGINI MARCATI CON INCHIOSTRO ) .",
       'CARCINOMA SCARSAMENTE DIFFERENZIATO AMPIAMENTE NECROTICO , NAS + D CATEGORIA C5 : POSITIVOD',
       "ADENOMA TUBULARE INTESTINALE ( 1 , 2 ) . 1 ) POLIPO A 4 CM . DALL ' ORIFIZIO ANALE . 2 ) POLIPO A 12 CM . DALL ' ORIFIZIO ANALE .",
       "OSTEOLISI ADDENSANTE FEMORE DISTALE CON INVASIONE PARTI MOLLI OD : OSTEOSARCOMA REPERTI ISTOLOGICI COMPATIBILI CON OSTEOSARCOMA AD ELEVATO GRADO DI MALIGNITA ' ISTOLOGICA . ESAME ISTOLOGICO AL CONGELATORE .",
       "NEGATIVO PER LESIONI INTRAEPITELIALI O MALIGNITA ' ( SISTEMA BETHESDA 2001 MOD . ) MATERIALE CITOLOGICO DA PRELIEVO PORTIO - ENDOCERVICE : NR . 1 VETRINO IN STRATO SOTTILE . ( ESEGUITA COLORAZIONE PAPANICOL

# write

In [109]:
pickle.dump(corpus, open(fileSaveCorpus, 'wb'))
pickle.dump(values, open(fileSaveValues, 'wb'))
pickle.dump(valuesM, open(fileSaveValuesM, 'wb'))
np.savetxt(fileSaveTrainWordVectors, wordVectorsText, delimiter=" ", fmt="%s")